In [8]:
# Libraries
import numpy as np
import scipy.linalg
from scipy.sparse import diags
import time
import sys, os
from math import sqrt

In [9]:
# Parameters
def inicial(sz = 4, tol = 1e-10):
    diags3 = np.array([-1 *np.ones(sz - 1),2 * np.ones(sz),-1 * np.ones(sz - 1)])
    config = [-1,0,1]
    A = diags(diags3,config).toarray()
    # A = np.array([[3.,1.,0.,0.], [1.,2.,3.,0], [0.,2.,2.,4], [0.,0.,5.,1.]])
    # A = np.array([[9.,5.,4.,3.],[9.,9.,2.,4.],[9.,9.,9.,8.],[9.,9.,9.,9.]])
    # A = 10*np.random.rand(5,5)
    print(A)
    return(A,sz,tol)

In [10]:
def j_rotation(M,p,l,k,n):
    A = M
    aDiff = A[l,l] - A[k,k]
    if abs(A[k,l]) < abs(aDiff)*1.0e-36: t = a[k,l]/aDiff
    else:
        tau0 = aDiff / (2.0 * M[k,l])
        t = 1.0 / (abs(tau0) + sqrt(tau0**2 + 1.0))
        if tau0 < 0.0: t = -t
    c = 1.0/sqrt(t**2 + 1.0); s = t*c
    tau = s/(1.0 + c)
    temp = A[k,l]
    A[k,l] = 0.0
    A[k,k] = A[k,k] - t*temp
    A[l,l] = A[l,l] + t*temp
    for i in range(k):
# Case of i < k
        temp = a[i,k]
        a[i,k] = temp - s*(a[i,l] + tau*temp)
        a[i,l] = a[i,l] + s*(temp - tau*a[i,l])
    for i in range(k+1,l):
# Case of k < i < l
        temp = a[k,i]
        a[k,i] = temp - s*(a[i,l] + tau*a[k,i])
        a[i,l] = a[i,l] + s*(temp - tau*a[i,l])
    for i in range(l+1,n):
# Case of i > l
        temp = a[k,i]
        a[k,i] = temp - s*(a[l,i] + tau*temp)
        a[l,i] = a[l,i] + s*(temp - tau*a[l,i])
    for i in range(n):
# Update transformation matrix
        temp = p[i,k]
        p[i,k] = temp - s*(p[i,l] + tau*p[i,k])
        p[i,l] = p[i,l] + s*(temp - tau*p[i,l])
    return(M,p)

In [11]:
# Search Function
def search_m(M,dim):
    Mval = 0
    for ii in range(0,dim-1):
        for jj in range(ii+1,dim):
            if abs(M[ii,jj]) > abs(Mval):
                Mval = M[ii,jj]
                l = ii
                k = jj
    return(Mval,l,k)

In [12]:
A,m,tol = inicial()

[[ 2. -1.  0.  0.]
 [-1.  2. -1.  0.]
 [ 0. -1.  2. -1.]
 [ 0.  0. -1.  2.]]


In [13]:
Mval,ll,kk = search_m(A,m)
a = A

In [14]:
p = np.identity(m,float)
for k in range(200):
    a,p= j_rotation(a,p,ll,kk,m)
print(a)
print(p)
#print("Jacobi method did not converge")

[[ 2.22044605e-16  1.12132034e+00 -7.07106781e-01  0.00000000e+00]
 [ 0.00000000e+00  3.12132034e+00 -7.07106781e-01  0.00000000e+00]
 [ 0.00000000e+00 -1.00000000e+00  2.00000000e+00 -1.00000000e+00]
 [ 0.00000000e+00  0.00000000e+00 -1.00000000e+00  2.00000000e+00]]
[[ 0.70710678 -0.70710678  0.          0.        ]
 [ 0.70710678  0.70710678  0.          0.        ]
 [ 0.          0.          1.          0.        ]
 [ 0.          0.          0.          1.        ]]


In [ ]:
akl = A[kk,ll]
akk = A[kk,kk]
all_= A[ll,ll]
tau = (all_ - akk) / (2 * akl)
t = 1 / (abs(tau) + sqrt(1 + tau ** 2))
if tau < 0.0: t = -t
c = 1 / sqrt(1 + tau **2)
s = c * t
A[kk,ll] = 0; A[ll,kk] = 0;
A[ll,ll]=all_*c*c-2*akl*c*s+akk*s*s;
A[kk,kk]=all_*s*s+2*akl*c*s+akk*c*c;
print(A)